In [1]:
//Nuget package installation
#r "nuget:Daany.DataFrame"
//Plot capabilities
#r "nuget: XPlot.Plotly.Interactive"

//Install Daany Linear algebra 
#r "nuget: Daany.LinA.win-x64"
    
//Linux version should install this package
//#r "nuget: Daany.LinA.linux-x64,1.0.0"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages Daany.DataFrame, 1.3.0 Daany.LinA.win-x64, 1.3.0 XPlot.Plotly.Interactive, 4.0.6

Loading extensions from `XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.

In [2]:

//local dlls
#r "Lib/FEMCommon.dll"
#r "Lib/FEMHeatLib.dll"
    
//define using statements
using System;
using System.IO;
using System.Linq;
using System.Text;
using System.Globalization;
using System.Collections.Generic;
using XPlot.Plotly;
using FEMHeat.Lib.BC;
using FEMCommon.Interfaces;
using FEMHeat.Lib.Solvers;
using FEMHeatLib.NModel;
using FEMCommon.Entities;
using Daany.MathStuff;
using Daany;
using FEMCommon.Helpers;
using System.Diagnostics;

# Numerical simulation of Quenching steel

The paper presents the numerical simulation of quenching cylindrical samples of steel. The quenching process starts from the initial temperature of the cylinder at 850 0C and then it moves through the air until it reaches the quenching bath. The immerse of the cylinder ends once the sample is completely in the quenchant. The quenchant is held at constant temperature of 40 0C. The process of quenching ends when the cylinder temperature reach the temperature of the quenchant. The cylinder is made of carful selected steel which do not change its structure during quenching. During the quenching process three measuring points were installed in the cylinder by measuring the temperatures beneath the cylinder surface at every half second. The process is recognized as transient and nonlinear heat transfer problem consisted of the two main tasks: direct heat transfer and inverse heat transfer problems. The paper presents methods used for the estimation of the temperature fields in time and heat transfer coefficients optimization of the inverse heat transfer problem. The second part of the paper presents the results and discussion of the quenching process. The results show that the optimization of the heat transfer coefficients are good enough to calculated the temperature field in time with error less than 3%.

Keywords: finite element method, Levenberg–Marquardt algorithm, quenching, multi-objective optimization, heat transfer coefficient.

## The source code content

The source code of the paper contains the implementetation of the several classes:
 - Material properties
 - Thermo couples measuring devices
 - Boundary and initial conditions
 - Experiment
 
All class implementation are used in the numerical calculation and optimization process for the calculation of temperature fields and heat transfer coenfficient. 

The following code implements the material properties of the cyluinder. The implementation also supported the termal coefficient k and cp change in function of temperatures. The regression curves k and cp in function of the temperatures are given based on the experimental data.

In [3]:
//Class implements material properties of the Cylinder
public class MHeat : IMData
{
    double _kx;
    double _rho;
    double _cp;
    public MHeat(double kx, double rho, double cp)
    {
        _kx = kx;
        _rho = rho;
        _cp = cp;
    }

    public double[] GetData(double? value)
    {
        //order of quantities
        //kx,ky,rho,cp
        if (value == null)
            return new double[4] { _kx, _kx, _rho, _cp };
        else
        {
            var l = -2E-6 * value.Value * value.Value + 0.0159 * value.Value + 14.712;
            var cp = 6E-7 * value.Value * value.Value * value.Value - 0.001 * value.Value * value.Value + 0.642 * value.Value + 436.57;
            return new double[4] { l,l, _rho, cp };
        }

    }
}

The thermo couples are measuring dvices located at different position in the cylinder. Depending of the position of thermo couples different htc values are calculated. The folowing code are suport thermo couplesa at 4 different positions.

In [4]:
//Class implements the Experiment by setting thermocouples at the specific location in the cylinder
public enum ThermoCouples
{
    Top,//corresponded to htc 3
    Middle,//corresponded to htc2
    Bottom,//corresponded to htc1
    OriginMiddle, //corresponded Tm3 thermocouples
}

The experiment class implementes specific position of mesuring points in the cylinder. There are 3 different experiment settings defined during the experiment phase.

In [5]:

public class Experiment
{
    public static PointD[] createTempSensor(double R, double H, ThermoCouples tc)
    {
        //setup dimension of cylinder and location of thermocouples
        var pts = new PointD[1];
        if (tc == ThermoCouples.Bottom)
           pts[0] = new PointD(Math.Round(R - 0.0015, 4), 0.0045);
        else if (tc == ThermoCouples.Middle)
           pts[0] = new PointD(Math.Round(R - 0.0015, 4), Math.Round(H / 2, 4));
        else
           pts[0] = new PointD(Math.Round(R - 0.0015, 4), Math.Round(H - 0.0045, 4));
        return pts;
    }
    public static PointD[] createTemp3Sensors(double R, double H)
    {
        //setup dimension of cylinder and location of thermocouples
        var pt1 = new PointD(Math.Round(R - 0.0015, 4), 0.0045);
        var pt2 = new PointD(Math.Round(R - 0.0015, 4), Math.Round(H / 2, 4));
        var pt3 = new PointD(Math.Round(R - 0.0015, 4), Math.Round(H - 0.0045, 4));
        var pts = new PointD[] { pt1, pt2, pt3 };
        return pts;
    }
    public static PointD[] createTemp4Sensors(double R, double H)
    {
        //setup dimension of cylinder and location of thermocouples
        var pt1 = new PointD(Math.Round(R - 0.0015, 4), 0.0045);
        var pt2 = new PointD(Math.Round(R - 0.0015, 4), Math.Round(H / 2, 4));
        var pt3 = new PointD(0, Math.Round(H / 2.0, 4));
        var pt4 = new PointD(Math.Round(R - 0.0015, 4), Math.Round(H - 0.0045, 4));
        var pts = new PointD[] { pt1, pt2, pt3, pt4 };
        return pts;
    }
}

The boundary conditions implements the values at the boundary domain. Since the proposed numerical model requires both 1D and 2D calculation domains, the two boundary condition classes are implemented: 
- 1D boundary conditions,
- 2D boundary conditions.

In [6]:
//Implementes 2D Axi-Symmetric Boundary Conditions
public class BoundaryConditions : IBoundaryConditions
{
public IMData _mprop;//Material property
public double R; 
public double H;//cylinder dimension
public PointD[] Tloc;//coordinate of thermocouples
public double Tin;//Initial temperature for starting time.
public float[] time { get; set; }//time 
public double[] h;//Height where thermocouples are located
public double[] htc1;
public double[] htc2;//htc for right cylinder surface 
public double[] htc3;//htc for top cylinder surface 
public double ta;//ambient temperature through time
public int TimeSteps { get; set; }

public static BoundaryConditions LoadFromFile(string filePath)
{
    var bc = new BoundaryConditions();
    var str = File.ReadAllLines(filePath);
    var lines = str.Where(x => x.Length > 1 && x[0] != '!').ToList();

    //Cylinder dimension
    var vals = lines[0].Split('\t', StringSplitOptions.RemoveEmptyEntries);
    (bc.R, bc.H) = (double.Parse(vals[0], CultureInfo.InvariantCulture), double.Parse(vals[1], CultureInfo.InvariantCulture));

    //initial temperature
    vals = lines[1].Split('\t', StringSplitOptions.RemoveEmptyEntries);
    bc.Tin = double.Parse(vals[0], CultureInfo.InvariantCulture);

    //number of time steps
    bc.TimeSteps = int.Parse(lines[2], CultureInfo.InvariantCulture);
    //time
    bc.time = lines[3].Split('\t', StringSplitOptions.RemoveEmptyEntries).Select(x => float.Parse(x, CultureInfo.InvariantCulture)).ToArray();

    //z coordinates of thermo couples
    bc.h = lines[4].Split('\t', StringSplitOptions.RemoveEmptyEntries).Select(x => double.Parse(x, CultureInfo.InvariantCulture)).ToArray();

    //ambient temperature through time
    bc.ta = lines[5].Split('\t', StringSplitOptions.RemoveEmptyEntries).Select(x => double.Parse(x, CultureInfo.InvariantCulture)).ToArray().First();

    //htcon bottom, right side and top sides of the cylinder
    bc.htc1 = lines[6].Split('\t', StringSplitOptions.RemoveEmptyEntries).Select(x => double.Parse(x, CultureInfo.InvariantCulture)).ToArray();
    bc.htc2 = lines[7].Split('\t', StringSplitOptions.RemoveEmptyEntries).Select(x => double.Parse(x, CultureInfo.InvariantCulture)).ToArray();
    bc.htc3 = lines[8].Split('\t', StringSplitOptions.RemoveEmptyEntries).Select(x => double.Parse(x, CultureInfo.InvariantCulture)).ToArray();

    return bc;
}
//Save BC to file. It is used when the optimization process completes to save the optimized values
public static void SaveToFile(BoundaryConditions bc, string filePath)
{
    if (bc == null)
        return;
    var str = new StringBuilder();
    str.AppendLine("!cylinder dimension RxH [m]");
    str.AppendLine($"{Math.Round(bc.R,4)}\t{Math.Round(bc.H, 4)}");
    str.AppendLine(" ");
    str.AppendLine("!initial temperature T0 [C]");
    str.AppendLine($"{Math.Round(bc.Tin, 0)}");
    str.AppendLine(" ");
    str.AppendLine("!number of timeSteps");
    str.AppendLine($"{bc.TimeSteps}");
    str.AppendLine(" ");
    str.AppendLine("! times [s]");
    str.AppendLine($"{string.Join('\t', bc.time)}");
    str.AppendLine(" ");
    str.AppendLine("!z coordinates of thermocouples");
    str.AppendLine($"{string.Join('\t', bc.Tloc.Select(x=>x.Y))}");
    str.AppendLine(" ");
    str.AppendLine("!ta [C] - ambient temperature");
    str.AppendLine($"{string.Join('\t', bc.ta)}");
    str.AppendLine(" ");
    str.AppendLine("!htc1, htc2, htc3 [W/(m^2K)]");
    str.AppendLine($"{string.Join('\t', bc.htc1.Select(x => Math.Round(x)))}");
    str.AppendLine($"{string.Join('\t', bc.htc2.Select(x => Math.Round(x)))}");
    str.AppendLine($"{string.Join('\t', bc.htc3.Select(x=> Math.Round(x)))}");

    File.WriteAllLines(filePath, str.ToString().Split(Environment.NewLine.ToCharArray(), StringSplitOptions.RemoveEmptyEntries));
}

public IBoundaryConditions Clone()
{
    var bc = new BoundaryConditions();
    bc.H = H;
    bc.h = h.Clone() as double[];
    bc.htc1 = htc1.Clone() as double[];
    bc.htc2 = htc2.Clone() as double[];
    bc.htc3 = htc3.Clone() as double[];
    bc.R= R;
    bc.ta = ta;
    bc.time = time.Clone() as float[];
    bc.TimeSteps = TimeSteps;
    bc.Tin= Tin;
    bc.Tloc = Tloc;
    bc._mprop = _mprop;
    return bc;
}
//Methods is called avery time the characteristics matrices is going to be calculate to setup BC
public IList<IBValue> GetBondaryConditions(IFiniteElement[] fe, INode[] nds, double[] prevNodeValues, int timeStep)
{
    //create bc list
    var bcs = new List<IBValue>();

    //for each finite element check each side of a triangle 
    for (int i = 0; i < fe.Length; i++)
    {
        var e = fe[i];
        //for all triangle side(1-2; 2-3; 3-1)
        for (int j = 0; j < e.N.Length; j++)
        {
            var side = j + 1;
            var n1 = nds[e.N[j]];//first node
            var n2 = j < e.N.Length - 1 ? nds[e.N[j + 1]] : nds[e.N[0]];//second node. 

            //check first side of a triangle
            var htc = getHTC(new PointD(n1.P.X, n1.P.Y), new PointD(n2.P.X, n2.P.Y), timeStep);
            if (Math.Abs(n1.P.Y - n2.P.Y) < 0.000001)
            {
                //set alpha 1
                if (n1.P.Y == 0)
                {
                    var bv = new HTBValue();
                    bv.Eid = e.Id;

                    bv.HTC.Add(side, (htc, ta));
                    bcs.Add(bv);
                }
                //set alpha 3
                if (Math.Abs(n1.P.Y - H) < 0.000001)
                {
                    var bv = new HTBValue();
                    bv.Eid = e.Id;
                    bv.HTC.Add(side, (htc, ta));
                    bcs.Add(bv);
                }
            }
            if (Math.Abs(n1.P.X - n2.P.X) < 0.000001 && Math.Abs(n1.P.X - R) < 0.000001)
            {
                //set alpha 2
                var bv = new HTBValue();
                bv.Eid = e.Id;
                bv.HTC.Add(side, (htc, ta));
                bcs.Add(bv);
            }
        }
    }

    return bcs;
}


public double getHTC(PointD pt1, PointD pt2, int timeStep)
{
    var r = pt1.X;
    var z = pt1.Y;

    if ((z == pt2.Y && (z == 0 || z == H)) || (r == pt2.X && r == R))
    {
        if (z <= H / 2.0)
            return (((htc2[timeStep] - htc1[timeStep]) / (h[1] - h[0])) * (z - h[0]) + htc1[timeStep]);
        else
            return (((htc3[timeStep] - htc2[timeStep]) / (h[2] - h[1])) * (z - h[1]) + htc2[timeStep]);
    }
    else
        return 0;
}
public double[] GetInitialValues(INode[] nds)
{
    return nds.Select(x=>Tin).ToArray();
}

public double? CalculateMDataParam(IFiniteElement[] fe, INode[] nds, double[] prevNodeValues, int timeStep)
{
    //return null;//this will consider constant values of kx, ky, rho and cp
    return prevNodeValues.Average();//this will take average temperature in the domain to calculate material properties
}

}

In [7]:
//Implementes Boundary Condition for 1D optimization problem
public class BoundaryConditions1D : IBoundaryConditions
{
public IMData _mprop;//Material property
public double R; 
public double H;//cylinder dimension
public PointD[] Tloc;//coordinate of thermocouples
public double Tin;//Initial temperature for starting time.
public float[] time { get; set; }//time 
public double[] htc;
public double ta;//ambient temperature through time
public int TimeSteps { get; set; }

public BoundaryConditions1D()
{ }
//convertor for 2D to 1D BC
public BoundaryConditions1D (BoundaryConditions bc2D,ThermoCouples tc)
{
    H = bc2D.H;
    if (tc == ThermoCouples.Bottom)
        htc = bc2D.htc1.Clone() as double[];
    else if (tc == ThermoCouples.Middle)
        htc = bc2D.htc2.Clone() as double[];
    else
        htc = bc2D.htc3.Clone() as double[];

    R = bc2D.R;
    ta = bc2D.ta;
    time = bc2D.time.Clone() as float[];
    TimeSteps = bc2D.TimeSteps;
    Tin = bc2D.Tin;
    Tloc = bc2D.Tloc;
    _mprop = bc2D._mprop;
}
public IBoundaryConditions Clone()
{
    var bc = new BoundaryConditions1D();
    bc.H = H;
    bc.htc = htc.Clone() as double[];    
    bc.R= R;
    bc.ta = ta;
    bc.time = time.Clone() as float[];
    bc.TimeSteps = TimeSteps;
    bc.Tin= Tin;
    bc.Tloc = Tloc;
    bc._mprop = _mprop;

    return bc;
}

public IList<IBValue> GetBondaryConditions(IFiniteElement[] fe, INode[] nds, double[] prevNodeValues, int timeStep)
{
    //create bc list
    var bcs = new List<IBValue>();

    //at last finite element put boundary condition
    var bv = new HTBValue();
    bv.Eid = fe.Last().Id;

    bv.HTC.Add(2, (htc[timeStep], ta));
    bcs.Add(bv);
    return bcs;
}

public double[] GetInitialValues(INode[] nds)
{
    return nds.Select(x=>Tin).ToArray();
}

public double? CalculateMDataParam(IFiniteElement[] fe, INode[] nds, double[] prevNodeValues, int timeStep)
{
    //return null;//this will consider constant values of kx, ky, rho and cp through time
    return prevNodeValues.Average();//this will take average temperature in the domain to calculate material properties
} 
}

### Loading experimental data

The starting point of the paper is loading the experimental data. The experimental data are presented at the paper (cite) and it is the starting point of the paper. The results are sconsisted of the measuring temperature at three points in the cylinder at every halph second. The following code loads the experimental data and stores in the application memory.

In [8]:
//Load measured temperatures 
public static List<(float time, double tm1, double tm2, double tm3, double tm4)> LoadTemperatures(string filePath)
{
    var str = File.ReadAllLines(filePath);
    var lines = str.Where(x => x.Length > 0 && x[0] != '!').ToList();
    var retVal = new List<(float time, double tm1, double tm2, double tm3, double tm4)>();
    foreach (var l in lines)
    {
        var ll = l.Split('\t', StringSplitOptions.RemoveEmptyEntries);
        if (ll.Length != 5)
            throw new Exception("File is not consistent.");
        var t = float.Parse(ll[0], CultureInfo.InvariantCulture);
        var t1 = double.Parse(ll[1], CultureInfo.InvariantCulture);
        var t2 = double.Parse(ll[2], CultureInfo.InvariantCulture);
        var t3 = double.Parse(ll[3], CultureInfo.InvariantCulture);
        var t4 = double.Parse(ll[4], CultureInfo.InvariantCulture);

        retVal.Add((t, t1, t2, t3, t4));
    }

    return retVal;
}


The paper proposes the solution of the Inverse Heat Transfer Problem byusing fist 1D and then 2D LM optimization algoritm, here the implementation of both optimization algorithm are presenteed.

In [9]:
public abstract class LMMBase
{
    protected double maxHtc = 35000.0;

    protected double epsilon = 0.05;


    protected double smallError = 0.000000001;
    protected double error1 = 5;
    protected double error2 = 0.5;



    protected double error1d1 = 1;
    protected double error1d2 = 1;

    protected abstract void chageHtc(IBoundaryConditions ibc, double dP, int timeStep, int htcIndex);
}

In [10]:
public class Inv1DLMMHTCSolverEx: LMMBase
{

    DHT _directSolver;
    List<(float time, double tm)> _Y;

    public Inv1DLMMHTCSolverEx(DHT directSOlver, List<(float time, double tm)> Y)
    {
        _directSolver = directSOlver;
        _Y = Y;
    }

    public double[] Optimize(BoundaryConditions1D bc, int Steps, bool verbose = false)
    {
        //add initial step
        int timeStep = 1;

        //solve direct problem for timeStep
        while (timeStep <= Steps)
        {
            var temps = _Y.Where(x => x.time == bc.time[timeStep]).FirstOrDefault();
            var y = temps.tm;
            solveInverseHTC(bc, timeStep, y);

            timeStep++;
        }

        if(verbose)
            Console.WriteLine($"1D optimization been completed.");

        return bc.htc.ToArray();
    }

    private void solveInverseHTC(BoundaryConditions1D bc, int timeStep, double Y)
    {

        double nu = 0.001;
        //calculate previous temperature field
        bc.TimeSteps = timeStep - 1;
        var tFieldInitial = _directSolver.Solve(bc);
        var prevField = tFieldInitial.Last().Value;
        var tempsInitial = _directSolver.Calculate(prevField, bc.Tloc[0]);
        var prevTemp = tempsInitial;


        //set initial value of htc to previous value
        double dP = bc.htc[timeStep-1];
        chageHtc(bc, dP, timeStep,0);

        //reset timeStep
        bc.TimeSteps = timeStep;

        //solve DHTP
        var bv1 = bc.GetBondaryConditions(_directSolver.Fes, _directSolver.Nodes, prevField, timeStep);
        var param1 = bc.CalculateMDataParam(_directSolver.Fes, _directSolver.Nodes, prevField, timeStep);
        var dt1 = bc.time[timeStep] - bc.time[timeStep - 1];

        //calculate temperature field
        var tField0 = _directSolver.Solve(prevField,bv1, dt1);
        var currTemp = _directSolver.Calculate(tField0, bc.Tloc[0]).First();


        //Calculate objective function
        var aev = currTemp - Y;  
        var SE0 = aev * aev;

        while (true)
        {

            //calculate sensitivity and Omega matrices
            var J = calculateSensitivity(tField0, currTemp, bc, timeStep);
            var O = J;            

            //Solve system of equation
            var lm = J + O * nu;
            var dm = J * aev;

            //calculate current increment for the heat transfer coefficient
            var currdP = dm/lm;

            //stop if dP is less than error2
            if (Math.Abs(currdP) < error1d2)
                break;

            //change htc for dP
            chageHtc(bc, currdP, timeStep,0);


            //DHTP (FEM)
            var bv = bc.GetBondaryConditions(_directSolver.Fes, _directSolver.Nodes, prevField, timeStep);
            var param = bc.CalculateMDataParam(_directSolver.Fes, _directSolver.Nodes, prevField, timeStep);
            var dt = bc.time[timeStep] - bc.time[timeStep - 1];

            var tField = _directSolver.Solve(prevField,bv, dt);
            currTemp = _directSolver.Calculate(tField, bc.Tloc[0]).First();

            //calculate objective function
            aev = currTemp - Y;
            var SP = aev * aev;

            //update dumping parameter          
            if (SP >= SE0)
                nu = 10 * nu;
            else
                nu = 0.1 * nu;

            //check stopping criteria
            if (SP < error1d1)
                return;

            SE0 = SP;

           //reset nu value if it is assigned to zero, or infinity
           if (nu == 0 || double.IsNaN(nu) || double.IsInfinity(nu))
             nu = 0.001;
        }

        return;
    }


    protected override void chageHtc(IBoundaryConditions ibc, double dP, int timeStep, int htcIndex)
    {
        BoundaryConditions1D bc = ibc as BoundaryConditions1D;
        var dh = 0.0;
        dh = bc.htc[timeStep] + dP;
        if (dh >= 0 && dh <= maxHtc)
            bc.htc[timeStep] = dh;     
    }

    private double calculateSensitivity(double[] tField0, double prevTemp, BoundaryConditions1D bc, int timeStep)
    {
        var bc1 = bc.Clone() as BoundaryConditions1D;
        var bc2 = bc.Clone() as BoundaryConditions1D;

        double eP1 = 0;
        eP1 = bc1.htc[timeStep] == 0 ? 100.0 * epsilon : bc1.htc[timeStep] * epsilon;
        bc1.htc[timeStep] -= eP1;

        //solve for P1 change
        var bv1 = bc1.GetBondaryConditions(_directSolver.Fes, _directSolver.Nodes, tField0, timeStep);
        var param1 = bc1.CalculateMDataParam(_directSolver.Fes, _directSolver.Nodes, tField0, timeStep);

        double eP2 = 0;
        eP2 = bc2.htc[timeStep] == 0 ? 100.0 * epsilon : bc1.htc[timeStep] * epsilon;
        bc2.htc[timeStep] += eP2;

        //solve for P1 change
        var bv2 = bc2.GetBondaryConditions(_directSolver.Fes, _directSolver.Nodes, tField0, timeStep);
        var param2 = bc2.CalculateMDataParam(_directSolver.Fes, _directSolver.Nodes, tField0, timeStep);

        var dt1 = bc1.time[timeStep] - bc1.time[timeStep - 1];
        var tfH1 = _directSolver.Solve(tField0,bv1, dt1);
        var tP1 = _directSolver.Calculate(tfH1, bc1.Tloc[0]).First();

        var dt2 = bc2.time[timeStep] - bc2.time[timeStep - 1];
        var tfH2 = _directSolver.Solve(tField0, bv2, dt2);
        var tP2 = _directSolver.Calculate(tfH2, bc2.Tloc[0]).First();

        return (tP2- tP1)/(2.0 * epsilon * eP1);

    }
}

Th eproposed solution in the paper requires first 1D optimization algoritm to find the values  near the optimal, and then run the 2D LM optimization algoritm starting from the previous results.  

In [11]:
public class Inv2DLMMHTSolverEx:LMMBase
{

DHT _directSolver;
List<(float time, double tm1, double tm2, double tm3)> _Y;

public Inv2DLMMHTSolverEx(DHT directSOlver, List<(float time, double tm1, double tm2, double tm3)> Y)
{
    _directSolver = directSOlver;
    _Y = Y;
}

public BoundaryConditions Optimize(BoundaryConditions bc, int Steps, bool verbose=false)
{
    //setup initial conditions
    double[] prevField = _directSolver.Nodes.Select(x => bc.Tin).ToArray(); 
    int timeStep = 1;

    if(verbose)
    {
        Console.WriteLine($"2D optimization has been started.");
        Console.WriteLine($" ");
    }

    //iterate trough the time
    while (timeStep <= Steps)
    {
        double[] y = null;
        var temps = _Y.Where(x => x.time == bc.time[timeStep]).FirstOrDefault();
        y = new double[3] { temps.tm1, temps.tm2, temps.tm3 };

        prevField = solveInverseHTC(prevField, bc, timeStep, y);

        Console.WriteLine($"Step {timeStep} of {Steps} completed.");
        //
        timeStep++;
    }
    Console.WriteLine($"2D optimization has completed.");
    return bc.Clone() as BoundaryConditions;
}


private double[] solveInverseHTC(double[] prevField, BoundaryConditions bc, int timeStep, double[] Y)
{
    double nu = 0.001;
    //reset time step
    bc.TimeSteps = timeStep;

    //set boundary condition for the current time step and solve direct problem
    var bv = bc.GetBondaryConditions(_directSolver.Fes,_directSolver.Nodes, prevField,timeStep);
    var param = bc.CalculateMDataParam(_directSolver.Fes, _directSolver.Nodes, prevField, timeStep);
    var dt = bc.time[timeStep] - bc.time[timeStep - 1];
    var tField = _directSolver.Solve(prevField, bv, dt);
    var currTemp = _directSolver.Calculate(tField, bc.Tloc);


    //calculation of objective function
    var aev = calculateAE(Y, currTemp);
    var SP = calculateSE(Y, currTemp);
    var OF0 = SP.Sum();

    //check stopping criteria
    if (OF0 < error1)
        return tField;

    //LMM iterartion
    while (true)
    {
        //Find the most dominant parameter
        var htcIndex = MaxArg(SP);

        //calculate sensitivity and Omega matrices
        var J = calculateSensitivity(prevField, bc, timeStep, htcIndex);
        var Jt = J.Transpose();
        var O = J.Dot(Jt)[0];

        //Solve system of equation
        var lm = J.Dot(Jt).Add(O * nu)[0];
        var dm = J.Dot(aev.Transpose())[0];

        //check result before proceed
        if (Math.Abs(dm) < smallError || dm == double.NaN || double.IsInfinity(dm) || lm == 0)
            break;

        //calculate current increment for the heat transfer coefficient
        var currdP = dm/lm;

        //check increment
        if (Math.Abs(currdP) < error2)
            break;

        //check the parameter accorind to previosu calculation
        chageHtc(bc, currdP, timeStep, htcIndex);


        //DHTP (AxiSymm FEM)
        var bv1 = bc.GetBondaryConditions(_directSolver.Fes, _directSolver.Nodes, prevField, timeStep);
        tField = _directSolver.Solve(prevField, bv1, dt);
        currTemp = _directSolver.Calculate(tField, bc.Tloc);

        //Calculate objective function
        aev = calculateAE(Y, currTemp);
        SP = calculateSE(Y, currTemp);
        var OF = SP.Sum();

        //update dumping parameter
        if (OF >= OF0)
            nu = 5 * nu;
        else
            nu = 0.5 * nu;

        //check stopping criteria
        if (OF < error1)
            break;

        OF0 = OF;
    }

    return tField;
}

private int MaxArg(double[] sp)
{
    return sp.ToList().IndexOf(sp.Max());
}

private double[] calculateSensitivity(double[] tField, BoundaryConditions bc, int timeStep, int htcIndex)
{

   // var epsilon = 0.05;
    var dt = bc.time[timeStep] - bc.time[timeStep - 1];

    //define P2 change
    var bc1 = bc.Clone() as BoundaryConditions;
    if (htcIndex == 0)
    {
        if (bc1.htc1[timeStep] == 0)
            bc1.htc1[timeStep] = 10 * epsilon;
        else
        {
            var eP21 = bc1.htc1[timeStep] * epsilon;
            bc1.htc1[timeStep] -= eP21;
        }
    }
    else if (htcIndex == 1)
    {
        if (bc1.htc2[timeStep] == 0)
            bc1.htc2[timeStep] = 10 * epsilon;
        else
        {
            var eP21 = bc1.htc2[timeStep] * epsilon;
            bc1.htc2[timeStep] -= eP21;
        }
    }
    else if (htcIndex == 2)
    {
        if (bc1.htc3[timeStep] == 0)
            bc1.htc3[timeStep] = 10 * epsilon;
        else
        {
            var eP21 = bc1.htc3[timeStep] * epsilon;
            bc1.htc3[timeStep] -= eP21;
        }
    }
    else
        throw new Exception("Unknown htc parameter.");


    //solve for P2 change
    var bv1 = bc1.GetBondaryConditions(_directSolver.Fes, _directSolver.Nodes, tField, timeStep);
    var param = bc1.CalculateMDataParam(_directSolver.Fes, _directSolver.Nodes, tField, timeStep);
    var tfH1 = _directSolver.Solve(tField, bv1, dt);
    var tP1 = _directSolver.Calculate(tfH1, bc1.Tloc);


    //define P2 change
    var bc2 = bc.Clone() as BoundaryConditions;
    if (htcIndex == 0)
    {
        if (bc2.htc1[timeStep] == 0)
            bc2.htc1[timeStep] = 10 * epsilon;
        else
        {
            var eP21 = bc2.htc1[timeStep] * epsilon;
            bc2.htc1[timeStep] += eP21;
        }
    }
    else if (htcIndex == 1)
    {
        if (bc2.htc2[timeStep] == 0)
            bc2.htc2[timeStep] = 10 * epsilon;
        else
        {
            var eP21 = bc2.htc2[timeStep] * epsilon;
            bc2.htc2[timeStep] += eP21;
        }
    }
    else if(htcIndex ==2)
    {
        if (bc2.htc3[timeStep] == 0)
            bc2.htc3[timeStep] = 10 * epsilon;
        else
        {
            var eP21 = bc2.htc3[timeStep] * epsilon;
            bc2.htc3[timeStep] += eP21;
        }
    }
    else
        throw new Exception("Unknown htc parameter.");

    //solve for P2 change
    var bv2 = bc2.GetBondaryConditions(_directSolver.Fes, _directSolver.Nodes, tField, timeStep);
    var tfH2 = _directSolver.Solve(tField, bv2, dt);
    var tP2 = _directSolver.Calculate(tfH2, bc2.Tloc);

    var dPj = 0.0;
    if (htcIndex == 0)
        dPj = bc.htc1[timeStep] == 0 ? 10 : bc.htc1[timeStep];
    else if (htcIndex == 1)
        dPj = bc.htc2[timeStep] == 0 ? 10 : bc.htc2[timeStep];
    else if (htcIndex == 2)
        dPj = bc.htc3[timeStep] == 0 ? 10 : bc.htc3[timeStep];
    else
        throw new Exception("Unknown htc parameter.");

    var J = tP2.Substruct(tP1).Divide(2.0 * epsilon * dPj);

    return J;

}


private double[] calculateSE(double[] measuredTemps, double[] calculatedTemps)
{
    var ae = measuredTemps.Zip(calculatedTemps, (first, second) => (first - second)*(first - second)).ToArray();
    return ae;
}


private double[] calculateAE(double[] measuredTemps, double[] calculatedTemps)
{
    var ae = measuredTemps.Zip(calculatedTemps, (first, second) => first - second).ToArray();
    return ae;
}


protected override void chageHtc(IBoundaryConditions ibc, double dP, int timeStep, int htcIndex)
{
    var bc = ibc as BoundaryConditions;
    if (htcIndex == 0)
    {
        var h1 = bc.htc1[timeStep] + dP;

        if (h1 >= 0 && h1 <= maxHtc)
            bc.htc1[timeStep] = h1;
    }
    else if (htcIndex == 1)
    {
        var h2 = bc.htc2[timeStep] + dP;

        if (h2 >= 0 && h2 <= maxHtc)
            bc.htc2[timeStep] = h2;
    }
    else if (htcIndex == 2)
    {
        var h3 = bc.htc3[timeStep] + dP;

        if (h3 >= 0 && h3 <= maxHtc)
            bc.htc3[timeStep] = h3;
    }
    else
        throw new Exception("Unknown parameter.");

}
}

The following implementation shows some helper funtions used in the proposed solution. 

In [12]:
private static void clearErrorTable()
{
    var esumm = DataFrame.FromCsv($"data/error_summary.txt");
    File.Delete($"data/error_summary.txt");
    var error = DataFrame.CreateEmpty(esumm.Columns);
    DataFrame.ToCsv($"data/error_summary.txt", error);
}

private static void runSimulation(string cylinderDim, string quenchant, int i)
{
    //lod boundary conditions
    var bc2D = BoundaryConditions.LoadFromFile($"data/{cylinderDim}/{quenchant}/bc.txt");

    //Load experimental measures of temperatures till defined timestep.
    var Y = LoadTemperatures($"data/{cylinderDim}/{quenchant}/t_data.txt").Where(x => x.time <= bc2D.time.Max()).ToList();

    //check ambient temperature
    var minTemp = Y.Select(x => Math.Min(Math.Min(Math.Min(x.tm1, x.tm2), x.tm3), x.tm4)).Min();
    if (minTemp < bc2D.ta)
        throw new Exception("Temperature of the quenchant cannot be lower than ambient temperature.");

    //Define material properties.
    IMData mdata = new MHeat(20.5, 7850, 460);//initial values of the properties
    bc2D._mprop = mdata;

    //Load 1D model
    var model1D = Model.LoadFromFile($"data/{ cylinderDim}/1d_model.txt");
    var directSolver = new DHT(model1D.fe, model1D.nds, mdata);

    //perform three independent optimizations

    //optimization at position 1
    var bc1 = new BoundaryConditions1D(bc2D, ThermoCouples.Bottom);
    bc1.Tloc = Experiment.createTempSensor(bc2D.R, bc2D.H, ThermoCouples.Bottom);
    var yy1 = Y.Select(x => (x.time, x.tm1)).ToList();

    //set initial values for multi-objective optimization 
    var htc1 = optimize1dProblem(model1D.fe, model1D.nds, directSolver, bc1, yy1, ThermoCouples.Bottom);

    //optimization at location 2
    var bc2 = new BoundaryConditions1D(bc2D, ThermoCouples.Middle);
    bc2.Tloc = Experiment.createTempSensor(bc2D.R, bc2D.H, ThermoCouples.Middle);
    var yy2 = Y.Select(x => (x.time, x.tm2)).ToList();

    //set initial values for multi-objective optimization 
    var htc2 = optimize1dProblem(model1D.fe, model1D.nds, directSolver, bc2, yy2, ThermoCouples.Middle);

    //optimization at location 3
    var bc3 = new BoundaryConditions1D(bc2D, ThermoCouples.Top);
    bc3.Tloc = Experiment.createTempSensor(bc2D.R, bc2D.H, ThermoCouples.Top);
    var yy3 = Y.Select(x => (x.time, x.tm4)).ToList();

    //set initial values for multi-objective optimization 
    var htc3 = optimize1dProblem(model1D.fe, model1D.nds, directSolver, bc3, yy3, ThermoCouples.Top);


    //define necessary measured points for multi-objective optimization
    bc2D.Tloc = Experiment.createTemp3Sensors(bc2D.R, bc2D.H);
    var yy2D = Y.Select(x => (x.time, x.tm1, x.tm2, x.tm4)).ToList();

    //prepare for multi-objective optimization
    var model2D = Model.LoadFromFile($"data/{cylinderDim}/2d_model.txt");
    var direct2DSolver = new DHT(model2D.fe, model2D.nds, mdata);
    bc2D.htc1 = htc1;
    bc2D.htc2 = htc2;
    bc2D.htc3 = htc3;
    var optBc = bc2D.Clone() as BoundaryConditions;
    int Steps = optBc.TimeSteps;
    optBc.TimeSteps = 0;

    //Multi-objective optimization
    var invSolver = new Inv2DLMMHTSolverEx(direct2DSolver, yy2D);
    var oBC = invSolver.Optimize(optBc, Steps);

    //post processing results
    resultPostprocessing(cylinderDim, quenchant, Y, direct2DSolver, oBC,i);
}

private static void resultPostprocessing(string cylinderDim, string quenchant, List<(float time, double tm1, double tm2, double tm3, double tm4)> Y, DHT direct2DSolver, BoundaryConditions oBC, int i)
{
    //calculate DHT and show results
    var optBC = oBC.Clone() as BoundaryConditions;
    optBC.Tloc = Experiment.createTemp4Sensors(optBC.R, optBC.H);
    var retVal = direct2DSolver.Solve(optBC);
    var temps = direct2DSolver.Calculate(retVal, optBC.Tloc);
    var ttime = optBC.time.Take(optBC.TimeSteps + 1);
    var dn1 = temps.Select(x => x.Value[0]).ToArray();
    var dm1 = Y.Where(x => ttime.Contains(x.time)).Select(x => x.tm1).ToArray();
    var dn2 = temps.Select(x => x.Value[1]).ToArray();
    var dm2 = Y.Where(x => ttime.Contains(x.time)).Select(x => x.tm2).ToArray();
    var dn3 = temps.Select(x => x.Value[2]).ToArray();
    var dm3 = Y.Where(x => ttime.Contains(x.time)).Select(x => x.tm3).ToArray();
    var dn4 = temps.Select(x => x.Value[3]).ToArray();
    var dm4 = Y.Where(x => ttime.Contains(x.time)).Select(x => x.tm4).ToArray();

    var dic = new Dictionary<string, List<object>>();
    dic.Add("t", ttime.Select(x => (object)x).ToList());
    dic.Add("TM1", dm1.Select(x => (object)x).ToList());
    dic.Add("TN1", dn1.Select(x => (object)x).ToList());
    dic.Add("TM2", dm2.Select(x => (object)x).ToList());
    dic.Add("TN2", dn2.Select(x => (object)x).ToList());
    dic.Add("TM3", dm3.Select(x => (object)x).ToList());
    dic.Add("TN3", dn3.Select(x => (object)x).ToList());
    dic.Add("TM4", dm4.Select(x => (object)x).ToList());
    dic.Add("TN4", dn4.Select(x => (object)x).ToList());



    //Save optimized values and results
    System.IO.DirectoryInfo di = new DirectoryInfo($"data/{cylinderDim}/{quenchant}/results/");
    foreach (FileInfo file in di.GetFiles())
    {
        file.Delete();
    }

    var temResults = new Daany.DataFrame(dic);
    var cols = new string[] { "R1", "R2", "R3", "R4", "RE1", "RE2", "RE3", "RE4" };

    temResults.AddCalculatedColumns(cols, (IDictionary<string, object> r, int i) =>
    {
        var tm1 = Convert.ToDouble(r["TM1"]);
        var tn1 = Convert.ToDouble(r["TN1"]);
        var tm2 = Convert.ToDouble(r["TM2"]);
        var tn2 = Convert.ToDouble(r["TN2"]);
        var tm3 = Convert.ToDouble(r["TM3"]);
        var tn3 = Convert.ToDouble(r["TN3"]);
        var tm4 = Convert.ToDouble(r["TM4"]);
        var tn4 = Convert.ToDouble(r["TN4"]);
        var r1 = Math.Abs(tm1 - tn1);
        var r2 = Math.Abs(tm2 - tn2);
        var r3 = Math.Abs(tm3 - tn3);
        var r4 = Math.Abs(tm4 - tn4);
        var re1 = r1 / tm1;
        var re2 = r2 / tm2;
        var re3 = r3 / tm3;
        var re4 = r4 / tm4;
        return new object[8] { r1, r2, r3, r4, re1, re2, re3, re4 };
    });

    //load error summary table and insert the error values for the current simulation
    var aetm1 = temResults["R1"].Select(x => Convert.ToDouble(x));
    var aetm2 = temResults["R2"].Select(x => Convert.ToDouble(x));
    var aetm3 = temResults["R3"].Select(x => Convert.ToDouble(x));
    var aetm4 = temResults["R4"].Select(x => Convert.ToDouble(x));
    var retm1 = temResults["RE1"].Select(x => Convert.ToDouble(x));
    var retm2 = temResults["RE2"].Select(x => Convert.ToDouble(x));
    var retm3 = temResults["RE3"].Select(x => Convert.ToDouble(x));
    var retm4 = temResults["RE4"].Select(x => Convert.ToDouble(x));

    var esumm = DataFrame.FromCsv($"data/error_summary.txt");
    File.Delete($"data/error_summary.txt");
    esumm.AddRow(new List<object>() {$"NS{i}", "Tm1", aetm1.Max(), aetm1.Average(), aetm1.Min(), retm1.Max(), retm1.Average(), retm1.Min()});
    esumm.AddRow(new List<object>() { $"NS{i}", "Tm2", aetm2.Max(), aetm2.Average(), aetm2.Min(), retm2.Max(), retm2.Average(), retm2.Min() });
    esumm.AddRow(new List<object>() { $"NS{i}", "Tm3", aetm3.Max(), aetm3.Average(), aetm3.Min(), retm3.Max(), retm3.Average(), retm3.Min() });
    esumm.AddRow(new List<object>() { $"NS{i}", "Tm4", aetm4.Max(), aetm4.Average(), aetm4.Min(), retm4.Max(), retm4.Average(), retm4.Min() });
    DataFrame.ToCsv($"data/error_summary.txt",esumm);

    DataFrame.ToCsv($"data/{cylinderDim}/{quenchant}/results/t_calculated.txt", temResults);
    BoundaryConditions.SaveToFile(optBC, $"data/{cylinderDim}/{quenchant}/results/htc_optimized.txt");
    Model.SaveToFile(direct2DSolver.Fes, direct2DSolver.Nodes, null, retVal, $"data/{cylinderDim}/{quenchant}/results/2d_model_results_{DateTime.Now.Ticks}.txt");
}

private static double[] optimize1dProblem(IFiniteElement[] elems, INode[] nodes, DHT directSol, BoundaryConditions1D bc, List<(float time, double tm)> Y, ThermoCouples tc)
{
    //Get the measured temperature
    var Ym = Y.Select(x => x.tm);
    var pt = bc.Tloc.First();
    //setup time steps
    int Steps = bc.TimeSteps;
    bc.TimeSteps = 0;//we start from initial time 

    var invSolver = new Inv1DLMMHTCSolverEx(directSol, Y);
    var htc = invSolver.Optimize(bc, Steps);
    return htc.ToArray();
}

private static void showResults(string cylinderDim, string quenchant)
{
    //Load experimental measures  of temperatures 
    var Y = LoadTemperatures($"data/{cylinderDim}/{quenchant}/t_data.txt");
    var bc = BoundaryConditions.LoadFromFile($"data/{cylinderDim}/{quenchant}/results/htc_optimized.txt");
    var Ye = DataFrame.FromCsv($"data/{cylinderDim}/{quenchant}/results/t_calculated.txt");
    var ttime = Ye["t"].Select(x => Convert.ToInt32(x));
    var tm1 = Ye["TM1"].Select(x => Convert.ToDouble(x));
    var tn1 = Ye["TN1"].Select(x => Convert.ToDouble(x));

    var tm2 = Ye["TM2"].Select(x => Convert.ToDouble(x));
    var tn2 = Ye["TN2"].Select(x => Convert.ToDouble(x));

    var tm3 = Ye["TM3"].Select(x => Convert.ToDouble(x));
    var tn3 = Ye["TN3"].Select(x => Convert.ToDouble(x));

    var tm4 = Ye["TM4"].Select(x => Convert.ToDouble(x));
    var tn4 = Ye["TN4"].Select(x => Convert.ToDouble(x));

    var l1 = new Line() { shape = "spline", color="black", width=0.3 };
    var l2 = new Line() { dash="dash", shape = "line", color = "black", width = 0.3 };
    var font = new Textfont() {family= "Times New Roman", size=13, color= "black", };
    var fontTitle = new Font() { family = "Times New Roman", color = "black", size = 16 };
    var font1 = new Font() { family = "Times New Roman", color = "black", size = 13 };

    var layout1 = new Layout.Layout()
    {
        titlefont = fontTitle,
        font = font1,
        xaxis = new Xaxis() { titlefont = font1, tickfont = font1,  showgrid=true },
        yaxis = new Yaxis() { titlefont = font1, tickfont = font1,  showgrid = true },
        width = 500,
        height = 350, legend= new Legend() { font=font1}

    };
    var layout2 = new Layout.Layout()
    {
        titlefont = fontTitle,
        font = font1,
        xaxis = new Xaxis() { titlefont = font1, tickfont = font1, showgrid = true },
        yaxis = new Yaxis() { titlefont = font1, tickfont = font1, showgrid = true },
        width = 500,
        height = 350,
        legend = new Legend() { font = font1 }

    };
    var layout3 = new Layout.Layout()
    {
        titlefont = fontTitle,
        font = font1,
        xaxis = new Xaxis() { titlefont = font1, tickfont = font1, showgrid = true },
        yaxis = new Yaxis() { titlefont = font1, tickfont = font1, showgrid = true },
        width=500, height=350,
        legend = new Legend() { font = font1 }
    };

    var circle01 = new Marker() { color = "white", symbol="circle", size = 7, line = new Line() { color = "black", width = 0.3 } };
    var circle = new Marker() { color = "black", symbol = "circle", size = 7, line = new Line() { color = "black", width = 0.3 } };
    var square = new Marker() { color = "black", symbol = "square", size = 7, line = new Line() { color = "black", width = 0.3 } };
    var diamond = new Marker() { color = "black", symbol = "diamond", size = 7, line = new Line() { color = "black", width = 0.3 } };
    var triangle = new Marker() { color = "black", symbol = "cross", size = 7, line = new Line() { color = "black", width = 0.3 } };

    var stm1 = new Scatter() {textfont=font, name = "TM 1", x = ttime, y = tm1, mode = "markers", marker=circle };
    var stn1 = new Scatter() {textfont=font, name = "TC 1", x = ttime, y = tn1, mode = "line", fillcolor = "black", line = l1 };
    var stm2 = new Scatter() {textfont=font, name = "TM 2", x = ttime, y = tm2, mode = "markers", marker = square };
    var stn2 = new Scatter() {textfont=font, name = "TC 2", x = ttime, y = tn2, mode = "line", fillcolor = "black", line = l1 };
    var stm3 = new Scatter() {textfont=font, name = "TM 3", x = ttime, y = tm3, mode = "markers", marker = diamond };
    var stn3 = new Scatter() {textfont=font, name = "TC 3", x = ttime, y = tn3, mode = "line", fillcolor = "black", line = l1 };
    var stm4 = new Scatter() {textfont=font, name = "TM 4", x = ttime, y = tm4, mode = "markers", marker = triangle };
    var stn4 = new Scatter() {textfont=font, name = "TC 4", x = ttime, y = tn4, mode = "line", fillcolor = "black", line = l1 };

    var chart1 = XPlot.Plotly.Chart.Plot<Scatter>(new Scatter[] { stm1, stn1, stm2, stn2, stm3, stn3, stm4, stn4 },layout1);
    chart1.WithTitle($"Numerical results for cylinder '{cylinderDim}' mm immersed in '{quenchant}'.");
    chart1.WithXTitle($"time, [sec]");
    chart1.WithYTitle($"temperature, [°C]");
    //chart1.Show();

    var line = new Scatter() {textfont=font, name = "", x = new double[] {0,850 }, y = new double[] { 0, 850 }, mode = "lines", line=l2 };
    var res1 = new Scatter() {textfont=font, name = "T1", x = tm1, y = tn1, mode = "markers", marker = circle01 };
    var res2 = new Scatter() {textfont=font, name = "T2",x = tm2, y = tn2, mode = "markers", marker = circle01 };
    var res3 = new Scatter() {textfont=font, name = "T3", x = tm3, y = tn3, mode = "markers", marker = circle01 };
    var res4 = new Scatter() {textfont = font, name = "T4", x = tm4, y = tn4, mode = "markers", marker = circle01 };

    var chart2 = XPlot.Plotly.Chart.Plot<Scatter>(new Scatter[] {line, res1,res2,res3,res4 },layout2);
    chart2.WithTitle($"Residual plot for cylinder '{cylinderDim}' mm immersed in '{quenchant}'.");
    chart2.WithXTitle($"measured temperature, [°C]");
    chart2.WithYTitle($"calculated temperature, [°C]");
    chart2.WithLegend(false);
    //chart2.Show();

    var htc1 = new Scatter() {textfont=font, name = "α1", x = ttime, y = bc.htc1, mode = "lines+markers", marker = circle, line = l1 };
    var htc2 = new Scatter() {textfont=font, name = "α2", x = ttime, y = bc.htc2, mode = "lines+markers", marker = diamond, line = l1 };
    var htc3 = new Scatter() {textfont = font, name = "α3", x = ttime, y = bc.htc3, mode = "lines+markers", marker = square, line = l1 };

    var chart3 = XPlot.Plotly.Chart.Plot<Scatter>(new Scatter[] { htc1, htc2, htc3 }, layout3);
    chart3.WithTitle($"Optimized HTCs for cylinder '{cylinderDim}' mm immersed in '{quenchant}'.");
    chart3.WithXTitle($"time, [sec]");
    chart3.WithYTitle($"HTC,α [W/m^2 K]");
    //chart3.Show();
    var lst = new List<XPlot.Plotly.PlotlyChart>() { chart1, chart2, chart3};
    Chart.ShowAll(lst);
}

Start of the algoritm after everything is prepared.

In [13]:
var cylinders = new string[] { "25x100"};//, "50x150", "75x225" };
var quenchants = new string[] {"H2O"};//, "Aquatensid5%", "Isorapid" };
var i = 1;

//clear error summary table
clearErrorTable();
var sw= Stopwatch.StartNew();
foreach (var cyl in cylinders)
{
    foreach(var quen in quenchants)
    {
        Console.WriteLine($"Numerical simulation for Cylinder:{cyl} immersed in {quen} has been started.");

        runSimulation(cyl, quen,i);

        Console.WriteLine($"Numerical simulation for Cylinder:{cyl} immersed in {quen} has completed.");
        Console.WriteLine($" ");
        i++;
    }
}
Console.WriteLine($"Numerical simulation took {sw.Elapsed.TotalMinutes}.");

Numerical simulation for Cylinder:25x100 immersed in H2O has been started.
Step 1 of 39 completed.
Step 2 of 39 completed.
Step 3 of 39 completed.
Step 4 of 39 completed.
Step 5 of 39 completed.
Step 6 of 39 completed.
Step 7 of 39 completed.
Step 8 of 39 completed.
Step 9 of 39 completed.
Step 10 of 39 completed.
Step 11 of 39 completed.
Step 12 of 39 completed.
Step 13 of 39 completed.
Step 14 of 39 completed.
Step 15 of 39 completed.
Step 16 of 39 completed.
Step 17 of 39 completed.
Step 18 of 39 completed.
Step 19 of 39 completed.
Step 20 of 39 completed.
Step 21 of 39 completed.
Step 22 of 39 completed.
Step 23 of 39 completed.
Step 24 of 39 completed.
Step 25 of 39 completed.
Step 26 of 39 completed.
Step 27 of 39 completed.
Step 28 of 39 completed.
Step 29 of 39 completed.
Step 30 of 39 completed.
Step 31 of 39 completed.
Step 32 of 39 completed.
Step 33 of 39 completed.
Step 34 of 39 completed.
Step 35 of 39 completed.
Step 36 of 39 completed.
Step 37 of 39 completed.
Step 38 o

# Results

In [14]:
var cylinders = new string[] { "25x100", "50x150", "75x225" };
var quenchants = new string[] {"H2O", "Aquatensid5%", "Isorapid" };

showResults(cylinders[0], quenchants[0]);
